In [1]:
#for interactive 3d plot 
!pip install PrettyTable > /dev/null 2>&1
from prettytable import PrettyTable
import sys
import math
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from mpl_toolkits.mplot3d import Axes3D
from sklearn import metrics
import enum
from sklearn import metrics
from scipy import stats
from sklearn.preprocessing import minmax_scale

# import ipyvolume as ipv
%matplotlib inline


The system cannot find the path specified.


In [2]:
# Files
# trainFile = 'trainNN.txt'
# testFile = 'testNN.txt'

train_dataset_file = 'vectorized_dataset_train.csv'
test_dataset_file = 'vectorized_dataset_test.csv'
dev_dataset_file = 'vectorized_dataset_dev.csv'

In [3]:
df_train = pd.read_csv(train_dataset_file, header = None)
df_test = pd.read_csv(test_dataset_file, header = None)
df_dev = pd.read_csv(dev_dataset_file, header = None)

x_train, y_train = df_train.iloc[:,:-1], df_train.iloc[:,-1]
x_test, y_test = df_test.iloc[:,:-1], df_test.iloc[:,-1]
x_dev, y_dev = df_dev.iloc[:,:-1], df_dev.iloc[:,-1]
# x_test, y_test = train_test_split(X,Y, test_size = 500/(700+500) ) 
# x_dev, y_dev = train_test_split(x_train,y_train, test_size = 200/(500+200) )
print(x_train.shape, x_test.shape, x_dev.shape)


(5500, 19998) (5500, 19998) (2200, 19998)


In [4]:

def castToType(x):
    try:
        return x.astype('float').astype('Int64') #pd 0.24+ 
    except:
        try:
            return x.astype('float')
        except:
            return x

def get_binarize_matrix(mat):
    tmp = np.copy(mat)
    tmp[tmp>0] = 1 
    return tmp 


class DataManager:
    def __init_(self, df):
        self.df = df.copy()
        self.df = df.applymap(lambda x: x.strip().lower() if isinstance(x, str) else x)
        self.df.replace(r'^[\s?]*$', np.nan, regex=True, inplace = True)
        self.df = self.df.apply(castToType)

    def get_handled_missing_values(self, nullThreshold = 0.7):

        for col in self.df.columns:
            if self.df[col].isnull().any() :
                if np.issubdtype(self.df[col].dtype, np.number):
                    self.df[col] = self.df[col].fillna(self.df[col].mean())
        
        
        self.df.dropna(thresh=self.df.shape[0]*nullThreshold,how='all',axis=1, inplace= True)
        self.df.dropna(axis = 0, inplace = True)
        
        return self.df

    def get_normalized_values(self):
        for i in range(len(self.df.columns)-1):
            self.df[:, i] = (self.df[:, i] - np.mean(self.df[:,i]))/np.std(self.df[:,i])
        
        return self.df
    
    def get_data_after_removed_cols(self):
        for col in self.df.columns:
            if len(self.df[col].unique()) == 1:
                print(col)
                df = self.df.drop(col,axis=1)

        for col in self.df.columns:
            if len(self.df[col].unique()) == len(self.df[col]):
                print(col)
                df = self.df.drop(col,inplace=True,axis=1)
        return df 
    
    def get_one_hot_encoded(df_column):
        from sklearn.preprocessing import LabelEncoder 
        from sklearn.preprocessing import OneHotEncoder 
        enc = OneHotEncoder(sparse=False, categories='auto')
        
        df_column = enc.fit_transform(df_column.reshape(len(df_column), -1))
        
        return df_column
        

In [5]:
from abc import ABC, abstractmethod 

class Distance:
    
    @abstractmethod
    def __call__(self, instance1, instance2):
        pass

#Distance calculation between two data points
class EuclideanDistance(Distance):
    def __init__(self):
        pass
    
    def __call__(self, instance1, instance2):
        instance1 = np.array(instance1)
        instance2 = np.array(instance2)
        return np.sqrt(np.sum((instance1 - instance2)**2) )

class HammingDistance(Distance):
    def __init__(self):
        pass
    
    def __call__(self, instance1, instance2):
        instance1 = np.array(instance1)
        instance2 = np.array(instance2)
        return np.sum(np.abs(instance1 - instance2)) 
    
class CosineSimilarity(Distance):
    def __init__(self):
        pass
    
    def __call__(self, instance1, instance2):
        instance1 = np.array(instance1)
        instance2 = np.array(instance2)
        return 1 - np.dot(instance1, instance2)/(np.linalg.norm(instance1)*np.linalg.norm(instance2))


class DISTANCE_ENUM(str, enum.Enum): 
    EUCLIDEAN = "euclidean"
    COSINE = "cosine"
    HAMMING = "hamming"
    

def get_distance(distance_func):
    if distance_func == DISTANCE_ENUM.EUCLIDEAN:
        return EuclideanDistance()
    elif distance_func == DISTANCE_ENUM.COSINE:
        return CosineSimilarity()
    elif distance_func == DISTANCE_ENUM.HAMMING:
        return HammingDistance()


In [6]:
    
class KNN:

    def __init__(self, n_neighbors = 3, weights='uniform', distance_func = 'cosine'):
        self.x_train = None
        self.y_train = None
        self.num_class = None
        self.weights = weights
        self.n_neighbors = n_neighbors
        self.distance_func_name = distance_func
        self.distance_func = get_distance(distance_func)
        self.total_train_data = 0
        self.x_train_tf_idf = []
        self.word_present_in_num_documents = []
        
    def fit(self, x_train, y_train):
        self.x_train = np.array(x_train, dtype='O')
        self.y_train = np.array(y_train, dtype='O')
        self.num_class = len(set(y_train))
        self.total_train_data = len(x_train) * 1.0
        x_train = np.array(x_train, dtype='O')
        x_train[x_train > 0] = 1
        self.word_present_in_num_documents = np.sum(x_train, axis=0) * 1.0
        self.x_train_tf_idf = self.get_TF_IDF(self.x_train)
        
    def get_TF_IDF(self, X_mat):
        tf = np.array(X_mat)
        tf = tf / tf.sum(axis=1, keepdims=True)
        idf = np.log(np.array(self.total_train_data / (self.word_present_in_num_documents + 1), dtype=float)  ) 
        return tf * idf 
        
    def predict(self, x_test):        
        allPredictedOutputs =[]
        x_test = np.array(x_test)
        distance_matrix = []
        if self.distance_func_name == DISTANCE_ENUM.COSINE:
            test_tf_idf = self.get_TF_IDF(x_test)
            distance_matrix = metrics.pairwise_distances(self.x_train_tf_idf, test_tf_idf, metric=self.distance_func_name )
        elif self.distance_func_name == DISTANCE_ENUM.HAMMING:
            distance_matrix = metrics.pairwise_distances(get_binarize_matrix(self.x_train),get_binarize_matrix(x_test), metric=self.distance_func_name )
        else:
            distance_matrix = metrics.pairwise_distances(self.x_train, x_test, metric=self.distance_func_name)

        #calculate for earch test data points
        for sample_no in range(len(x_test)):

            allDistances = np.vstack(
                (self.y_train,
                distance_matrix[:,sample_no])
            ).T
            

            allDistances = allDistances[allDistances[:,-1].argsort()] 


            #Assuming output labels are from 0 to uniqueOutputCount-1
            voteCount = dict()
            neighbors = []
            for i in range(self.n_neighbors):
                voting_weight = 1.0 / ( allDistances[i][1] + 1e-8 )  # 1 / distance 
                class_label = int(allDistances[i][0])
                voteCount[class_label] = voting_weight if class_label not in voteCount else voteCount[class_label] + voting_weight
                
            #Determine the Majority Voting (Equal weight considered)
            mxCnt, mxClass = 0, 0
            for key in voteCount:
                if voteCount[key] > mxCnt:
                    mxCnt, mxClass = voteCount[key] , key

            allPredictedOutputs.append(mxClass)
        return allPredictedOutputs   #, allTestNeighbers


In [7]:
t = PrettyTable(['Method', 'K', 'Accuracy'])

best_distance_func = ''
best_k_in_Knn = 1
best_accuracy = 0

for dist_func in DISTANCE_ENUM:
    for k in [1,3,5]:
        model = KNN(n_neighbors = k, distance_func = dist_func)
        model.fit(x_train, y_train)
        y_pred = model.predict(x_dev)
        accuracy = metrics.accuracy_score(y_dev, y_pred) * 100
        print(dist_func, k,  accuracy)
        
        if accuracy >  best_accuracy:
            best_distance_func, best_k_in_Knn = dist_func, k
            best_accuracy = accuracy
        t.add_row([dist_func, k,  accuracy])
            

print(best_accuracy, best_k_in_Knn, best_distance_func)

DISTANCE_ENUM.EUCLIDEAN 1 57.45454545454546
DISTANCE_ENUM.EUCLIDEAN 3 57.77272727272727
DISTANCE_ENUM.EUCLIDEAN 5 56.54545454545455
DISTANCE_ENUM.COSINE 1 81.22727272727272
DISTANCE_ENUM.COSINE 3 83.45454545454545
DISTANCE_ENUM.COSINE 5 83.5
DISTANCE_ENUM.HAMMING 1 39.86363636363636
DISTANCE_ENUM.HAMMING 3 41.27272727272727
DISTANCE_ENUM.HAMMING 5 40.54545454545455
83.5 5 DISTANCE_ENUM.COSINE


In [8]:
with open('1505014_report.txt', 'w') as f:
    f.write('KNN\n' )
    f.write('Accurary for Validation set\n')
    f.write(t.get_string())
    f.write('\n')

In [9]:


        
def getReplacedList(y):
        class_list = list(set(sorted(y)))

        class_dict_transform = {}

        i = 0
        for original_class in class_list:
            class_dict_transform[original_class] = i
            i+=1

        for i in range(len(y)):
            y[i] = class_dict_transform[y[i]]
        return y

class NaiveBayes:

    def __init__(self, smoothing_factor = 1.0):

        self.class_list = None
        self.n = 0 
        self.classwise_summary = []
        self.class_frequency = []
        self.num_feature = 0
        self.smoothing_factor = float(smoothing_factor)

    
    def fit(self, x_train, y_train):
        x_train = np.array(x_train)
        y_train = np.array(y_train)
        self.n = len(x_train) * 1.0
        self.num_feature = len(x_train[0]) 
        self.class_list = list(set(sorted(y_train))) 
        y_train = getReplacedList(y_train)
        
        classwise_mat = [ [] for _ in range (len(self.class_list)) ]    # 2d matrix grouped by classes 
        self.class_frequency = [ 0 ] * len(self.class_list)
        
        
        for x,y in zip(x_train, y_train):
            classwise_mat[y].append(x)
            self.class_frequency[y] += 1
        
        self.class_frequency = np.array(self.class_frequency) / self.n * 100.0
        
        self.P_w_given_Class = []
        
        total_word_in_entire_train = np.sum(x_train)
        
        for class_label in range(len(self.class_list)):
            
            mat = np.array(classwise_mat[class_label])
            total_words_under_class = np.sum(mat) 
            self.P_w_given_Class.append( np.sum(mat, axis=0) + self.smoothing_factor / \
                            (total_words_under_class + self.smoothing_factor * total_word_in_entire_train ) )
        
    def predict(self, x_test): 
        y_pred = []
        x_test_binary = np.copy(x_test)
        x_test_binary[x_test_binary > 0] = 1.0
#         print(self.class_frequency.shape)
        for i in range(len(x_test_binary)):
            x = x_test_binary[i,:].reshape(1, self.num_feature)
            P_Dt_given_class = x * self.P_w_given_Class 

#             cnt = np.count_nonzero(x!=0)
#             P_Dt_given_class = P_Dt_given_class[P_Dt_given_class!=0].reshape(-1,cnt)
            P_Dt_given_class[ P_Dt_given_class == 0 ] = 1
            P_Dt_given_class = np.prod(P_Dt_given_class, axis = 1)
            posterior_prob = self.class_frequency * P_Dt_given_class
            pred_class = np.argmax(posterior_prob )
            
            y_pred.append(self.class_list[pred_class])

        return y_pred     


In [10]:


t = PrettyTable(['SF', 'Accuracy'])

best_sf_naive = 1
best_accuracy = 0

for sf in np.linspace(0.1, 1, num=10):
    model = NaiveBayes(smoothing_factor=sf)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_dev)
    accuracy = metrics.accuracy_score(y_dev, y_pred) * 100
    print(sf,  accuracy)
    
    if accuracy >  best_accuracy:
        best_sf_naive = sf
        best_accuracy = accuracy
    t.add_row([sf, accuracy])


0.1 84.31818181818181
0.2 84.27272727272728
0.30000000000000004 84.36363636363636
0.4 84.36363636363636
0.5 84.36363636363636
0.6 84.36363636363636
0.7000000000000001 84.31818181818181
0.8 84.31818181818181
0.9 84.31818181818181
1.0 84.31818181818181


In [11]:
with open('1505014_report.txt', 'a') as f:
    f.write('Naive Bayes\n' )
    f.write('Accurary for Validation set\n')
    f.write(t.get_string())
    f.write('\n')

## Test data accuracy with best hyperparamters

In [12]:
def get_stats(ara):
    return pd.DataFrame(np.array(ara)).describe().loc[['min','max', 'std']]

In [13]:
num_test_iterations = 50
knn_test_accuracies = []

t = PrettyTable(['Iteration No', 'Accuracy'])
print( best_k_in_Knn, best_distance_func)
model = KNN(n_neighbors = best_k_in_Knn, distance_func = best_distance_func)
model.fit(x_train, y_train)
for i in range(num_test_iterations):

    y_pred = model.predict(x_test.iloc[i*10:(i+1)*10, :])
    accuracy = metrics.accuracy_score(y_test.iloc[i*10:(i+1)*10], y_pred) * 100
    print(accuracy)
    knn_test_accuracies.append(accuracy)
    t.add_row([i,  accuracy])


5 DISTANCE_ENUM.COSINE
100.0
100.0
90.0
70.0
80.0
80.0
50.0
50.0
90.0
90.0
90.0
80.0
60.0
80.0
100.0
90.0
90.0
70.0
60.0
90.0
100.0
90.0
90.0
90.0
90.0
80.0
90.0
70.0
90.0
70.0
80.0
90.0
70.0
100.0
90.0
80.0
80.0
80.0
70.0
100.0
60.0
90.0
70.0
70.0
70.0
80.0
90.0
60.0
70.0
90.0


In [14]:
with open('1505014_report.txt', 'a') as f:
    f.write('KNN\n' )
    f.write('Accurary for Test set\n')
    f.write(t.get_string())
    f.write('\n')

In [15]:
num_test_iterations = 50
naive_test_accuracies = []

t = PrettyTable(['Iteration No', 'Accuracy'])
print('Best smoothing factor ', best_sf_naive)
model = NaiveBayes(smoothing_factor=best_sf_naive)
model.fit(x_train, y_train)
for i in range(num_test_iterations):

    y_pred = model.predict(x_test.iloc[i*10:(i+1)*10, :])
    accuracy = metrics.accuracy_score(y_test.iloc[i*10:(i+1)*10], y_pred) * 100
    print(accuracy)
    naive_test_accuracies.append(accuracy)
    t.add_row([i,  accuracy])

Best smoothing factor  0.30000000000000004
100.0
100.0
100.0
100.0
80.0
90.0
80.0
80.0


C:\Users\HP\Anaconda3\envs\torch\lib\site-packages\numpy\core\fromnumeric.py:87: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


90.0
90.0
100.0
80.0
90.0
60.0
100.0
90.0
90.0
90.0
100.0
100.0
90.0
100.0
100.0
90.0
80.0
90.0
80.0
80.0
100.0
90.0
60.0
90.0
90.0
100.0
90.0
80.0
100.0
90.0
80.0
90.0
80.0
70.0
90.0
70.0
80.0
80.0
100.0
90.0
70.0
100.0


In [16]:
with open('1505014_report.txt', 'a') as f:
    f.write('Naive Bayes\n' )
    f.write('Accurary for Test set\n')
    f.write(t.get_string())
    f.write('\n')

In [17]:
stats.ttest_rel(knn_test_accuracies,naive_test_accuracies)

Ttest_relResult(statistic=-3.6421413166111147, pvalue=0.0006515010281061904)

In [18]:
get_stats(knn_test_accuracies)

,0
min,50.000000
max,100.000000
std,13.192453


In [19]:
get_stats(naive_test_accuracies)

,0
min,60.000000
max,100.000000
std,10.631106


In [20]:
# neigh.predict(x_test)
# print(metrics.accuracy_score(y_test,neigh.predict(x_test)) * 100)


In [21]:
# a = {'4': 'ac', '34':'45'}
# lst = [4,5,34,5,55,5,4]
# class_list = list(set(sorted(lst)))

# tmp = getReplacedList(lst)

# lst2 = [3,1,1,1,0]